# <font color=#025dfa> Giyim Eşyalarını Tanıma - Identifying Clothing Items

TR = Her yorum satırı kendisini üstündeki koda aittir. İlk olarak Türkçe, son olarak İngilizce yazıldı.

EN = Each comment line belongs to the code above it. It was first written in Turkish and lastly in English.

TR = Giyim eşyalarını görüntüler üzerinden otomatik olarak tanıyan ve sınıflandıran bir model geliştirmeyi amaçlayan bu proje, derin öğrenme ve bilgisayarla görme teknikleri kullanarak farklı giysi türlerini analiz edecektir.

EN = Aiming to develop a model that automatically recognizes and classifies clothing items from images, this project will analyze different types of clothing using deep learning and computer vision techniques.

Kaynak/Source = https://thecleverprogrammer.com/2020/08/30/image-classification-with-tensorflow-in-machine-learning/

In [1]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Conv2D, InputLayer , Reshape , MaxPooling2D, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import VGG16,ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator #Tek satırda resimleri okumayı sağlıyor


In [2]:
from tensorflow import keras
from keras.utils import to_categorical
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# TR = Etiketleri saklayacak bir liste oluşturur.
# EN = Creates a list to store the labels.


## <font color=#FFD700> EDA Keşif Amaçlı Veri Analizi - EDA - Exploratory Data Analysis

In [4]:
print('x_train=',x_train.shape)
print('x_test=',x_test.shape)
print('y_train=',y_train.shape)
print('y_test=',y_test.shape)

x_train= (60000, 28, 28)
x_test= (10000, 28, 28)
y_train= (60000,)
y_test= (10000,)


In [ ]:
for label_index, label_name in enumerate(labels):
    # TR = İlgili etiketle eşleşen tüm görüntü yolları bir listeye alınır  
    # EN = All image paths matching the label are retrieved into a list
    label_images = x_train[y_train == label_index]  
    
    if len(label_images) >= 9:  
        # TR = Eğer görüntü sayısı 9 veya daha fazlaysa işlem devam eder  
        # EN = Proceed if there are 9 or more images

        selected_images = random.sample(list(label_images), 9)  
        # TR = Görüntüler arasından rastgele 9 tanesi seçilir  
        # EN = Randomly select 9 images from the list
        
        plt.figure(figsize=(15, 15))  
        # TR = 15x15 boyutlarında bir figür oluşturulur  
        # EN = Create a figure of size 15x15
        
        for i, img in enumerate(selected_images):  
            # TR = Seçilen her bir görüntü için döngü başlatılır  
            # EN = Loop through each selected image
            
            plt.subplot(3, 3, i + 1)  
            # TR = 3x3'lük bir yerleşimde görüntü konumlandırılır  
            # EN = Place the image in a 3x3 grid
            
            plt.imshow(img, cmap='gray')  
            # TR = Görüntü ekranda gösterilir  
            # EN = Display the image in grayscale
            
            plt.title(label_name)  
            # TR = Her görüntüye ait başlık olarak etiket eklenir  
            # EN = Set the label as the title for each image
            
            plt.axis('off')  
            # TR = Eksenler gizlenir  
            # EN = Hide the axes
        
        plt.show()  
        # TR = Görüntüler ekranda gösterilir  
        # EN = Display the images on the screen
        
    else:  
        # TR = Eğer yeterli görüntü yoksa uyarı verir  
        # EN = If not enough images are available, show a warning message
        print(f"Yeterli görüntü bulunamadı/Not enough images found: {label_name}")  
        # TR = Ekranda yeterli görüntü bulunamadığını belirtir  
        # EN = Print a message that not enough images were found

In [5]:
x_train=x_train/255.0
x_test=x_test/255.0

In [6]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

## <font color='#0F52BA'> Öznitelik Mühendisliği - Feature Engineering

### <font color=#007fff> Model - Modelling

In [7]:
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=.20,random_state=42)
# TR = modelimizi eğittik.
# EN = We trained our model.

In [8]:
size=28
model = Sequential()
# TR = Ardışık bir model oluşturur, katmanlar sıralı olarak eklenir.
# EN = Creates a sequential model where layers are added in a linear stack.

model.add(InputLayer(input_shape=(size,size,1)))
# TR = Modelin giriş katmanını tanımlar ve veri boyutunu belirtir.
# EN = Defines the input layer of the model and specifies the data dimensions.

model.add(Reshape(target_shape=(size,size,1)))
# TR = Veriyi 28x28x1 şeklinde yeniden düzenler, gri tonlama görüntüleri için tek bir renk kanalını belirtir.
# EN = Reshapes the data to 28x28x1, specifying a single color channel for grayscale images.

model.add(Conv2D(filters=12, kernel_size=(3,3), activation='relu'))
# TR = Konvolüsyon işlemi ile özellikleri çıkarır, filtreler görüntüdeki desenleri öğrenir ve 'relu' aktivasyon fonksiyonu ile doğrusal olmayan ilişkileri modellemesini sağlar.
# EN = Extracts features via convolution, where filters learn patterns in the image and the 'relu' activation function introduces non-linearity.

model.add(MaxPooling2D(pool_size=(2,2)))
# TR = Özellik haritasını küçültür ve en belirgin özellikleri seçer, böylece hesaplama maliyeti azalır ve modelin genelleştirme yeteneği artar.
# EN = Reduces the feature map size and selects the most prominent features, reducing computation and improving the model’s ability to generalize.

model.add(Flatten())
# TR = Çok boyutlu veri kümesini tek boyutlu bir vektöre dönüştürür, tam bağlantılı katmanlarla işlem için uygun hale getirir.
# EN = Converts the multi-dimensional feature map into a one-dimensional vector to be processed by fully connected layers.

model.add(Dense(10))
# TR = Sonuçları 10 farklı sınıfa dönüştüren bir tam bağlantılı katman ekler, bu da modelin sınıflandırma yapmasını sağlar.
# EN = Adds a fully connected layer with 10 units to output results for 10 different classes, enabling classification.

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# TR = Modeli 'adam' optimizasyonu ile derler, bu optimizasyon algoritması öğrenmeyi iyileştirir, ve 'SparseCategoricalCrossentropy' kayıp fonksiyonunu kullanarak çok sınıflı sınıflandırma yapar, doğruluk metriğiyle model performansını değerlendirir.
# EN = Compiles the model with the 'adam' optimizer, which improves learning, uses 'SparseCategoricalCrossentropy' loss function for multi-class classification, and evaluates performance with accuracy metric.

model.fit(x_train, y_train, validation_split=.10, epochs=10)
# TR = Modeli eğitim verileri (train_images) ve etiketleri (train_labels) ile eğitir, %10'luk bir doğrulama seti kullanır ve 10 dönem boyunca eğitir.
# EN = Trains the model with the training data (train_images) and labels (train_labels), uses 10% of the data for validation, and trains for 10 epochs.

Epoch 1/10
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7638 - loss: 0.7150 - val_accuracy: 0.8492 - val_loss: 0.4170
Epoch 2/10
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8673 - loss: 0.3781 - val_accuracy: 0.8698 - val_loss: 0.3556
Epoch 3/10
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8839 - loss: 0.3355 - val_accuracy: 0.8794 - val_loss: 0.3345
Epoch 4/10
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8918 - loss: 0.3088 - val_accuracy: 0.8854 - val_loss: 0.3184
Epoch 5/10
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8976 - loss: 0.2893 - val_accuracy: 0.8929 - val_loss: 0.3083
Epoch 6/10
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9062 - loss: 0.2689 - val_accuracy: 0.8925 - val_loss: 0.3060
Epoch 7/10
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9089 - loss: 0.2620 - val_accuracy: 0.8950 - val_loss: 0.2957
Epoch 8/10
1350/1350 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9120 - loss: 0.2532 -

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ reshape (Reshape)                    │ (None, 28, 28, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 26, 26, 12)          │             120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 12)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2028)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │          20,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 61,232 (239.19 KB)

 Trainable params: 20,410 (79.73 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 40,822 (159.46 KB)

In [10]:
loss, _accuracy = model.evaluate(x_test, y_test)
# TR = Test verileri (test_images) ve etiketleri (test_labels) ile modelin performansını değerlendirir, kayıp ve doğruluk değerlerini döndürür.
# EN = Evaluates the model's performance using test data (test_images) and labels (test_labels), returning loss and accuracy values.

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9006 - loss: 0.2761


## <font color='#ec11f7'> Transfer Learning

In [11]:
import numpy as np
import cv2
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# TR = Fashion MNIST veri setini yüklüyoruz. Veriler eğitim ve test olarak ikiye ayrılır.
# EN = We load the Fashion MNIST dataset. The data is split into training and testing sets.

train_images = np.stack([train_images] * 3, axis=-1)
test_images = np.stack([test_images] * 3, axis=-1)
# TR = Gri tonlamalı görüntüleri üç kanal olarak yığınlıyoruz, böylece görüntüler RGB formatında olur.
# EN = We stack grayscale images into three channels, making the images in RGB format.

size = 32

train_images = np.array([cv2.resize(image, (size, size)) for image in train_images])
test_images = np.array([cv2.resize(image, (size, size)) for image in test_images])
# TR = Görüntüleri yeniden boyutlandırıyoruz, her birini 32x32 piksel boyutuna getiriyoruz.
# EN = We resize the images, making each 32x32 pixels.

train_images = train_images / 255.0
test_images = test_images / 255.0
# TR = Görüntüleri normalize ederek piksel değerlerini 0 ile 1 arasında bir aralığa getiriyoruz.
# EN = We normalize the images, scaling pixel values between 0 and 1.

train_datagen = ImageDataGenerator(validation_split=0.2)
# TR = Eğitim verileri için %20 doğrulama verisi ayırarak bir veri üreticisi oluşturuyoruz.
# EN = We create a data generator for the training data, reserving 20% for validation.

train_datagenerator = train_datagen.flow(train_images, train_labels, batch_size=32, subset='training')
# TR = Eğitim verileri üzerinde veri üreticisi oluşturuyoruz.
# EN = We create a data generator on the training data.

validation_datagenerator = train_datagen.flow(train_images, train_labels, batch_size=32, subset='validation')
# TR = Doğrulama verileri üzerinde veri üreticisi oluşturuyoruz.
# EN = We create a data generator on the validation data.

base_model = VGG16(weights='imagenet', input_shape=(size, size, 3), include_top=False)
# TR = VGG16 önceden eğitilmiş modelini kullanıyoruz, son katmanı hariç tutuyoruz.
# EN = We use the VGG16 pre-trained model, excluding the top (final) layer.

for layer in base_model.layers:
  layer.trainable = False
# TR = VGG16 modelindeki tüm katmanların eğitimde güncellenmesini devre dışı bırakıyoruz.
# EN = We freeze all layers in the VGG16 model to prevent them from being updated during training.

model = Sequential()
model.add(base_model)

model.add(Flatten())
# TR = Base modeli ve düzleştirme katmanını modelimize ekliyoruz.
# EN = We add the base model and a flattening layer to our model.

model.add(Dense(1024, activation='relu'))
# TR = Bir tam bağlantılı katman (1024 nöron) ekliyoruz.
# EN = We add a fully connected layer with 1024 neurons.

model.add(Dense(10, activation='softmax'))
# TR = Çıkış katmanını ekliyoruz (10 sınıf için softmax aktivasyonu).
# EN = We add the output layer with softmax activation for 10 classes.

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# TR = Modeli Adam optimizasyonu, sparse categorical crossentropy kaybı ve doğruluk metriği ile derliyoruz.
# EN = We compile the model with Adam optimizer, sparse categorical crossentropy loss, and accuracy metric.

history=model.fit(train_datagenerator, epochs=10, validation_data=validation_datagenerator)
# TR = Modeli 10 epoch boyunca eğitiyoruz, doğrulama verileri ile doğruluğu ölçüyoruz.
# EN = We train the model for 10 epochs, measuring accuracy on the validation data.

loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')
# TR = Test verileri üzerinde modelin kaybını ve doğruluğunu değerlendiriyoruz.
# EN = We evaluate the model's loss and accuracy on the test data.

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.7723 - loss: 0.6346 - val_accuracy: 0.8438 - val_loss: 0.4217
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.8459 - loss: 0.4112 - val_accuracy: 0.8543 - val_loss: 0.3863
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.8640 - loss: 0.3679 - val_accuracy: 0.8637 - val_loss: 0.3689
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.8751 - loss: 0.3367 - val_accuracy: 0.8658 - val_loss: 0.3622
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.8827 - loss: 0.3121 - val_accuracy: 0.8671 - val_loss: 0.3577
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.8873 - loss: 0.2978 - val_accuracy: 0.8728 - val_loss: 0.3457
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.8926 - loss: 0.2827 - val_accuracy: 0.8720 - val_loss: 0.3568
Epoch 8/10
1500/1500 ━━━━━━━━━━━

In [11]:
plt.plot(history.history['accuracy'],label='Accuracy')
plt.plot(history.history['val_accuracy'],label='Val_Accuracy')
plt.legend();